In [11]:
import pandas as pd
import re
from datetime import datetime

In [12]:
df = pd.read_csv("orders.csv")
df_ab = pd.read_csv('abandoned.csv')

df.columns

Index(['Name', 'Email', 'Financial Status', 'Paid at', 'Fulfillment Status',
       'Fulfilled at', 'Accepts Marketing', 'Currency', 'Subtotal', 'Shipping',
       'Taxes', 'Total', 'Discount Code', 'Discount Amount', 'Shipping Method',
       'Created at', 'Lineitem quantity', 'Lineitem name', 'Lineitem price',
       'Lineitem compare at price', 'Lineitem sku',
       'Lineitem requires shipping', 'Lineitem taxable',
       'Lineitem fulfillment status', 'Billing Name', 'Billing Street',
       'Billing Address1', 'Billing Address2', 'Billing Company',
       'Billing City', 'Billing Zip', 'Billing Province', 'Billing Country',
       'Billing Phone', 'Shipping Name', 'Shipping Street',
       'Shipping Address1', 'Shipping Address2', 'Shipping Company',
       'Shipping City', 'Shipping Zip', 'Shipping Province',
       'Shipping Country', 'Shipping Phone', 'Notes', 'Note Attributes',
       'Cancelled at', 'Payment Method', 'Payment Reference',
       'Refunded Amount', 'Vendor', '

# make order table

In [13]:
def Make_Order_Table(df):
    order = df[
        [    'Name','Created at','Financial Status',
            'Paid at', 'Fulfillment Status', 
             'Fulfilled at','Accepts Marketing', 'Currency', 'Subtotal', 'Shipping',
             'Taxes','Total', 'Discount Code', 'Discount Amount', 'Shipping Method',
             'Billing Name', 'Billing Address1', 'Billing Address2', 
             'Billing City', 'Billing Zip', 'Billing Province', 'Billing Country',
             'Billing Phone', 'Shipping Name', 'Shipping Address1', 
             'Shipping Address2', 'Shipping City', 'Shipping Zip',
             'Shipping Province', 'Shipping Country', 'Shipping Phone',
             'Notes','Payment Method', 'Email'
    ]
    ].copy()

    #Remove duplicate values
    order['Name'] = order['Name'].str.replace("#", "")
    order['Name'] = order["Name"].drop_duplicates()
    order = order[order["Name"].notna()]
    
    #Map Proper names
    order.rename(columns =
                    {
                        'Name':'order_id',
                        'Created at':'created_at',
                        'Financial Status':'financial_status',
                        'Paid at':'paid_at',
                        'Fulfillment Status':'fulfillment_status', 
                        'Fulfilled at':'fulfilled_at',
                        'Accepts Marketing':'accepts_marketing',
                        'Currency':'currency',
                        'Subtotal':'subtotal',
                        'Shipping':'shipping',
                        'Taxes':'taxes',
                        'Total':'total',
                        'Discount Code':'discount_code',
                        'Discount Amount':'discount_amount',
                        'Shipping Method':'shipping_method',
                        'Billing Name': 'billing_name',
                        'Billing Address1':'billing_address1',
                        'Billing Address2':'billing_address2',
                        'Billing City':'billing_city',
                        'Billing Zip':'billing_zip',
                        'Billing Province':'billing_province',
                        'Billing Country': 'billing_country',
                        'Billing Phone':'billing_phone',
                        'Shipping Name':'shipping_name',
                        'Shipping Address1' : 'shipping_address1',
                        'Shipping Address2' : 'shipping_address2',
                        'Shipping City':'shipping_city',
                        'Shipping Zip':'shipping_zip',
                        'Shipping Province':'shipping_province',
                        'Shipping Country':'shipping_country',
                        'Shipping Phone':'shipping_phone', 
                        'Notes':'notes',
                        'Payment Method':'payment_method',    
                        'Email':'email'
                    }, 
                    
                    inplace = True
                   )
    order.reset_index(drop = True, inplace = True)

    # Convert to datetime object and remove timezone info
    order['created_at'] = pd.to_datetime(order['created_at']).dt.tz_localize(None)
    order['paid_at'] = pd.to_datetime(order['paid_at']).dt.tz_localize(None)
    order['fulfilled_at'] = pd.to_datetime(order['paid_at']).dt.tz_localize(None)
    
    # Format as string in desired format
    order['created_at'] = order['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
    order['paid_at'] = order['paid_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
    order['fulfilled_at'] = order['fulfilled_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # Fill NaN values
    order['created_at'] = order['created_at'].fillna('2020-01-01 00:00:00')
    order['paid_at'] = order['paid_at'].fillna('2020-01-01 00:00:00')
    order['fulfilled_at'] = order['fulfilled_at'].fillna('2020-01-01 00:00:00')
    order.fillna('null', inplace = True)
    
    return order

# Customer Table

In [14]:
def Make_Customer_Table(df):
    customer = df[
        ['Email', 'Billing Name', 'Billing Province']
    ].copy()
    
    customer['Email'] = customer["Email"].drop_duplicates()
    customer = customer[customer["Email"].notna()]
    customer.rename(columns = {
                            'Email':'email',
                            'Billing Name':'full_name',
                            'Billing Province':'location'
                    },                    
                    inplace=True
                   )
    
    customer.fillna('null', inplace = True)
   
    customer.reset_index(drop=True, inplace=True)   
    return customer

# Product Table

In [15]:
def Make_Product_Table(df):
    product = df[
        ['Lineitem sku', 'Lineitem name', 'Lineitem price',
         'Lineitem compare at price','Lineitem requires shipping',
         'Lineitem taxable']
    ].copy()
    
    product['Lineitem name'] = product["Lineitem name"].drop_duplicates()
    product = product[product["Lineitem name"].notna()]
    product['Lineitem price'] = product["Lineitem price"].drop_duplicates()
   
    product = product[product["Lineitem price"].notna()]   
    product.fillna('null', inplace = True)

    #Map Proper names
    product.rename(columns =
                    {
                        'Lineitem sku':'product_sku',
                        'Lineitem name':'product_name',
                        'Lineitem price':'product_price',
                        'Lineitem compare at price':'product_compare_at_price',
                        'Lineitem requires shipping':'product_requires_shipping', 
                        'Lineitem taxable':'product_taxable',                      
                    },
                   
                   inplace = True
                  )  
    #drop null
    product = product[product["product_sku"] != 'null']
    
    product.reset_index(drop = True, inplace = True)
        
    return product

# Order Item

In [16]:
def Make_Order_Item_Table(df):
    order_item = df[
        ['Name', 'Lineitem sku', 'Lineitem quantity', 'Lineitem fulfillment status']
    ].copy()
    
    order_item['Name'] = order_item['Name'].str.replace("#", "")
    
    order_item.fillna('null', inplace = True)
    
    order_item.rename(columns =
                    {
                      'Name':'order_id',
                      'Lineitem sku':'product_sku',
                      'Lineitem quantity':'quantity',
                      'Lineitem fulfillment status':'product_fulfillment'                
                    },
    
                   inplace = True
                  )  
    
    order_item.reset_index(drop = True, inplace = True)
    
    return order_item


# Abandoned_order

In [17]:
abandoned_order = Make_Order_Table(df_ab)
abandoned_order.rename(columns = {'order_id':'abandoned_order_id'}, inplace = True)
abandoned_order.drop(['financial_status', 'paid_at',
                      'fulfillment_status', 'fulfilled_at', 
                      'payment_method'],
                     axis = 1,
                     inplace = True 
                    )

C:\Users\armen\anaconda3\lib\site-packages\pandas\core\indexes\extension.py:101: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(result, name=self.name)


# Abandoned_order_item

In [18]:
abandoned_order_item = Make_Order_Item_Table(df_ab)
abandoned_order_item.rename(columns = {'order_id':'abandoned_order_id'}, inplace = True)

In [19]:
order_item = Make_Order_Item_Table(df)
product = Make_Product_Table(df)
customer = Make_Customer_Table(df)
order = Make_Order_Table(df)

# Transfer to DB

In [22]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(user='root', password='arm2002',
                        host='10.100.101.109', database='coolina',
                        auth_plugin='mysql_native_password')    
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        
        # Insert into customer
        
        n = 1
        
        for i,row in customer.iterrows():
            sql = ("INSERT IGNORE INTO coolina.customer "
                   "(email, full_name, location) "
                   "VALUES (%s, %s, %s)"
                  )
            cursor.execute(sql, tuple(row))
            print(f"{n} Record inserted Customer")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
            n += 1
            
        
        # Insert into orders
        
        n = 1
        
        for i,row in order.iterrows():
            sql = ("INSERT IGNORE INTO coolina.orders "
                   "(order_id, created_at, financial_status, paid_at,"
               "fulfillment_status, fulfilled_at, accepts_marketing, currency,"
               "subtotal, shipping, taxes, total, discount_code,"
               "discount_amount, shipping_method, billing_name,"
               "billing_address1, billing_address2, billing_city, billing_zip,"
               "billing_province, billing_country, billing_phone, shipping_name,"
               "shipping_address1, shipping_address2, shipping_city,"
               "shipping_zip, shipping_province, shipping_country,"
               "shipping_phone, notes, payment_method, email)"
                   "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,"
                   " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                   )
            cursor.execute(sql, tuple(row))
            print(f"{n} Record inserted to ORDERS")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
            n += 1            
                    
        # Insert into Abandoned orders
        
        n = 1
        
        for i,row in abandoned_order.iterrows():
            sql = ("INSERT IGNORE INTO coolina.abandoned_order "
                   "(abandoned_order_id, created_at, accepts_marketing, currency,"
               "subtotal, shipping, taxes, total, discount_code,"
               "discount_amount, shipping_method, billing_name,"
               "billing_address1, billing_address2, billing_city, billing_zip,"
               "billing_province, billing_country, billing_phone, shipping_name,"
               "shipping_address1, shipping_address2, shipping_city,"
               "shipping_zip, shipping_province, shipping_country,"
               "shipping_phone, notes, email)"
                   "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,"
                   " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                   )
            cursor.execute(sql, tuple(row))
            print(f"{n} Record inserted to ABANDONED ORDERS")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
            n += 1    
            
         
        # Insert into product
        n = 1
        
        for i,row in product.iterrows():
            sql = ("INSERT IGNORE INTO coolina.product "
            "(product_sku, product_name, product_price, product_compare_at_price, product_requires_shipping, product_taxable) "
            "VALUES (%s, %s, %s, %s, %s, %s)"
             )
            cursor.execute(sql, tuple(row))
            print(f"{n} Record inserted Product")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
            n += 1
         
        # Insert into order_item
        n = 1
        
        for i,row in order_item.iterrows():
            sql = ("INSERT IGNORE INTO coolina.order_item "
            "(order_id, product_sku, quantity, product_fulfillment)"                
            "VALUES (%s, %s, %s, %s)"
             )
            cursor.execute(sql, tuple(row))
            print(f"{n} Record inserted Order Item")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
            n += 1
                   
       # Insert into abandoned_order
        n = 1
        
        for i,row in abandoned_order_item.iterrows():
            sql = ("INSERT IGNORE INTO coolina.abandoned_order_item "
            "(abandoned_order_id, product_sku, quantity, product_fulfillment)"                
            "VALUES (%s, %s, %s, %s)"
             )
            cursor.execute(sql, tuple(row))
            print(f"{n} Record inserted Abandoned Order Item")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
            n += 1
            
            
except Error as e:
    print("Error while connecting to MySQL", e)

You're connected to database:  ('coolina',)
1 Record inserted Customer
2 Record inserted Customer
3 Record inserted Customer
4 Record inserted Customer
5 Record inserted Customer
6 Record inserted Customer
7 Record inserted Customer
8 Record inserted Customer
9 Record inserted Customer
10 Record inserted Customer
11 Record inserted Customer
12 Record inserted Customer
13 Record inserted Customer
14 Record inserted Customer
15 Record inserted Customer
16 Record inserted Customer
17 Record inserted Customer
18 Record inserted Customer
19 Record inserted Customer
20 Record inserted Customer
21 Record inserted Customer
22 Record inserted Customer
23 Record inserted Customer
24 Record inserted Customer
25 Record inserted Customer
26 Record inserted Customer
27 Record inserted Customer
28 Record inserted Customer
29 Record inserted Customer
30 Record inserted Customer
31 Record inserted Customer
32 Record inserted Customer
33 Record inserted Customer
34 Record inserted Customer
35 Record ins

152 Record inserted Order Item
153 Record inserted Order Item
154 Record inserted Order Item
1 Record inserted Abandoned Order Item
2 Record inserted Abandoned Order Item
3 Record inserted Abandoned Order Item
4 Record inserted Abandoned Order Item
5 Record inserted Abandoned Order Item
6 Record inserted Abandoned Order Item
7 Record inserted Abandoned Order Item
8 Record inserted Abandoned Order Item
9 Record inserted Abandoned Order Item
10 Record inserted Abandoned Order Item
11 Record inserted Abandoned Order Item
12 Record inserted Abandoned Order Item
13 Record inserted Abandoned Order Item
14 Record inserted Abandoned Order Item
15 Record inserted Abandoned Order Item
16 Record inserted Abandoned Order Item
17 Record inserted Abandoned Order Item
18 Record inserted Abandoned Order Item
19 Record inserted Abandoned Order Item
20 Record inserted Abandoned Order Item
21 Record inserted Abandoned Order Item
22 Record inserted Abandoned Order Item
23 Record inserted Abandoned Order I